In [1]:
# Técnicas de Ensemble

# Bagging e Random Forest utilizando o framework tidymodels (parsnip, baguette).
# Instalando e carregando pacotes necessários
install.packages('baguette')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘plyr’, ‘reshape2’, ‘libcoin’, ‘mvtnorm’, ‘Formula’, ‘inum’, ‘listenv’, ‘parallelly’, ‘warp’, ‘globals’, ‘sparsevctrs’, ‘lobstr’, ‘Cubist’, ‘partykit’, ‘DiceDesign’, ‘sfd’, ‘future’, ‘slider’, ‘parsnip’, ‘butcher’, ‘C50’, ‘dials’, ‘furrr’, ‘hardhat’, ‘rsample’




In [2]:
install.packages('ranger')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘RcppEigen’




In [3]:
library(baguette)
library(ranger)

Loading required package: parsnip



In [5]:
dados <- read.csv('automoveis_usados.csv')

In [6]:
head(dados)

,Estilo,Ano,Potencia_motor,Cilindros_motor,Consumo_estrada_km,Valor
,<chr>,<int>,<dbl>,<dbl>,<dbl>,<int>
1,SUV 4 portas,2005,275,6,28.96812,29695
2,seda,2016,170,4,61.15492,30495
3,minivan de passageiros,2016,248,6,45.06152,37650
4,seda,2015,138,4,57.93624,16170
5,seda,1991,162,4,32.18680,2000
6,picape cabine estendida,2012,152,4,37.01482,19299


In [7]:
install.packages('tidymodels')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘shape’, ‘future.apply’, ‘numDeriv’, ‘progressr’, ‘SQUAREM’, ‘diagram’, ‘lava’, ‘prodlim’, ‘lhs’, ‘patchwork’, ‘clock’, ‘gower’, ‘ipred’, ‘timeDate’, ‘GPfit’, ‘modelenv’, ‘infer’, ‘modeldata’, ‘recipes’, ‘tailor’, ‘tune’, ‘workflows’, ‘workflowsets’, ‘yardstick’




In [8]:
library(tidymodels)

── Attaching packages ────────────────────────────────────── tidymodels 1.4.1 ──

✔ broom        1.0.10     ✔ rsample      1.3.1 
✔ dials        1.4.2      ✔ tailor       0.1.0 
✔ dplyr        1.1.4      ✔ tidyr        1.3.1 
✔ ggplot2      4.0.0      ✔ tune         2.0.0 
✔ infer        1.0.9      ✔ workflows    1.3.0 
✔ modeldata    1.5.1      ✔ workflowsets 1.1.1 
✔ purrr        1.1.0      ✔ yardstick    1.3.2 
✔ recipes      1.3.1      

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ purrr::discard() masks scales::discard()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()
✖ recipes::step()  masks stats::step()



In [9]:
set.seed(6578)

df_split <- initial_split(dados, prop = 0.8)
df_split

<Training/Testing/Total>
<8655/2164/10819>

In [10]:
df_treino <- training(df_split)
df_teste <- testing(df_split)

In [11]:
# BAGGING

# - bag_tree(): cria um modelo de árvore de decisão com ensemble
# - set_mode('regression'): indica que estamos tratando um problema de regressão
# - set_engine('rpart', times = 100): usa o motor 'rpart' com 100 árvores no ensemble
bagging <- bag_tree() %>%
  set_mode('regression') %>%
  set_engine('rpart', times = 100)

In [12]:
# Ajustando o modelo Bagging nos dados de treino
set.seed(4556)

modelo_bagging <- bagging %>%
  fit(Valor ~ Estilo + Ano + Potencia_motor, data = df_treino)

Registered S3 method overwritten by 'butcher':
  method                 from    
  as.character.dev_topic generics



In [13]:
# Visualizando informações do modelo ajustado
modelo_bagging

parsnip model object

Bagged CART (regression with 100 members)

Variable importance scores include:

# A tibble: 3 × 4
  term             value   std.error  used
  <chr>            <dbl>       <dbl> <int>
1 Ano            1.24e12 2188886368.   100
2 Potencia_motor 9.77e11 2166065457.   100
3 Estilo         3.19e11 1250656068.   100


In [14]:
# Fazendo previsões no conjunto de teste
predicoes <- predict(modelo_bagging, new_data = df_teste)

In [15]:
# Unindo as previsões com os dados de teste para facilitar análise
predicoes_badding <- bind_cols(df_teste, predicoes)

In [16]:
predicoes_badding

Estilo,Ano,Potencia_motor,Cilindros_motor,Consumo_estrada_km,Valor,.pred
<chr>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>
hatchback 2 portas,2015,106,4,59.54558,14845,16056.09
seda,2008,264,6,38.62416,25495,28489.01
hatchback 2 portas,2005,210,4,45.06152,23670,25001.69
perua,1991,162,4,32.18680,2000,2000.00
SUV 4 portas,2017,168,4,46.67086,23600,24951.54
SUV 4 portas,2017,284,6,43.45218,36100,36703.74
SUV 2 portas,2001,203,6,32.18680,21035,23570.72
hatchback 4 portas,2016,138,4,59.54558,17905,18102.96
hatchback 2 portas,2013,200,4,53.10822,31495,27757.42


In [17]:
calcular_metricas <- function(predicoes, valores_reais) {
  resultado <- tibble(
    predicao = predicoes,
    real = valores_reais
  )

  metricas <- resultado %>%
    metrics(truth = real, estimate = predicao)
  return(metricas)
}

In [19]:
set.seed(4556)

df_folds <- vfold_cv(df_treino, v = 5)

In [18]:
# Calculando métricas de desempenho do Bagging:
# - RMSE (Root Mean Squared Error)
# - R² (Coeficiente de Determinação)
# - MAE (Mean Absolute Error)
metricas_badding <- calcular_metricas(predicoes_badding$.pred, predicoes_badding$Valor)
metricas_badding

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
rmse,standard,4562.5555314
rsq,standard,0.9173159
mae,standard,3022.3345081


In [20]:
# Avaliando a performance do modelo Bagging com validação cruzada (resampling)
set.seed(4456)

fits_cv <- fit_resamples(bagging,
                         Valor ~ Estilo + Ano + Potencia_motor,
                         resamples = df_folds,
                         metrics = metric_set(rmse, mae, rsq))

In [21]:
# Coletando as métricas médias da validação cruzada
media_metricas_badding <- collect_metrics(fits_cv)
media_metricas_badding

.metric,.estimator,mean,n,std_err,.config
<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
mae,standard,2975.1605054,5,34.638275543,pre0_mod0_post0
rmse,standard,4575.2697907,5,83.031434098,pre0_mod0_post0
rsq,standard,0.9180205,5,0.002730385,pre0_mod0_post0


In [22]:
# RANDOM FOREST

# Instalando e carregando o pacote necessário
install.packages('randomForest')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [23]:
library(randomForest)

randomForest 4.7-1.2

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:ggplot2’:

    margin


The following object is masked from ‘package:dplyr’:

    combine


The following object is masked from ‘package:ranger’:

    importance




In [25]:
# Definição do modelo Random Forest:
# - rand_forest(): cria um modelo de floresta aleatória
# - trees = 100: número de árvores
# - set_mode('regression'): regressão
# - set_engine('ranger'): usa o motor ranger (rápido e eficiente)
floresta_aleatoria <- rand_forest(trees = 100) %>%
  set_mode('regression') %>%
  set_engine('ranger')

In [26]:
# Ajustando o modelo Random Forest
set.seed(4456)

modelo_floresta_aleatoria <- floresta_aleatoria %>%
  fit(Valor ~ Estilo + Ano + Potencia_motor, data = df_treino)

In [27]:
# Visualizando informações do modelo treinado
modelo_floresta_aleatoria

parsnip model object

Ranger result

Call:
 ranger::ranger(x = maybe_data_frame(x), y = y, num.trees = ~100,      num.threads = 1, verbose = FALSE, seed = sample.int(10^5,          1)) 

Type:                             Regression 
Number of trees:                  100 
Sample size:                      8655 
Number of independent variables:  3 
Mtry:                             1 
Target node size:                 5 
Variable importance mode:         none 
Splitrule:                        variance 
OOB prediction error (MSE):       27633249 
R squared (OOB):                  0.891849 

In [28]:
# Fazendo previsões no conjunto de teste
predicoes <- predict(modelo_floresta_aleatoria, new_data = df_teste)

In [29]:
predicoes_floresta <- bind_cols(df_teste, predicoes)

In [30]:
# Calculando métricas de desempenho do Random Forest
metricas_floresta <- calcular_metricas(predicoes_floresta$.pred, predicoes_floresta$Valor)
metricas_floresta

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
rmse,standard,5288.1087367
rsq,standard,0.8930113
mae,standard,3596.6924404


In [31]:
# Avaliando a performance do modelo Random Forest com validação cruzada
floresta_aleatoria <- rand_forest(trees = 100, mtry = 2 ) %>%
  set_mode('regression') %>%
  set_engine('ranger')

In [32]:
set.seed(4456)

fits_cv <- fit_resamples(floresta_aleatoria,
                         Valor ~ Estilo + Ano + Potencia_motor,
                         resamples = df_folds,
                         metrics = metric_set(rmse, mae, rsq))

In [33]:
# Coletando métricas médias da validação cruzada
media_metricas_floresta <- collect_metrics(fits_cv)
media_metricas_floresta


.metric,.estimator,mean,n,std_err,.config
<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
mae,standard,2966.1068016,5,34.234648351,pre0_mod0_post0
rmse,standard,4529.4666225,5,71.117077538,pre0_mod0_post0
rsq,standard,0.9196171,5,0.002152886,pre0_mod0_post0
